In [16]:
import wave
import struct
import os

In [17]:
from pvrecorder import PvRecorder

for index, device in enumerate(PvRecorder.get_available_devices()):
    print(f"[{index}] {device}")

[0] 外置麦克风
[1] MacBook Pro麦克风
[2] LarkAudioDevice
[3] “Zilv233”的麦克风
[4] Microsoft Teams Audio


In [21]:
# 用户语音输入
recorder = PvRecorder(device_index=-1, frame_length=512)
audio = []

path = f"./test.mp3"

try:
    recorder.start()

    while True:
        frame = recorder.read()
        audio.extend(frame)
except KeyboardInterrupt:
    recorder.stop()
    with wave.open(path, 'w') as f:
        f.setparams((1, 2, 16000, 512, "NONE", "NONE"))
        f.writeframes(struct.pack("h" * len(audio), *audio))
finally:
    recorder.delete()

In [12]:
# cheimu_dev
# TODO: 输入用户秘钥
os.environ["OPENAI_API_KEY"] = 

In [19]:
from openai import OpenAI
client = OpenAI()

In [22]:
# 识别语音
audio_file= open(path, "rb")
translation = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file,
  language="zh"
)
print(translation.text)

现在需要你给出一个场地为20米长20米的咖啡馆环境的设计,要求列出环境内存在哪些物品,物品摆放的坐标和物品,根据其使用环境的缩放系数,以及根据其形态和功能决定的物体朝向,返回数据的格式为一个表格。


In [24]:
# gpt生成场景
gpt_model = "gpt-3.5-turbo"

completion = client.chat.completions.create(
  model=gpt_model,
  messages=[
    {"role": "system", "content": "你是一名使用UE5引擎进行关卡开发的游戏内环境设计师，擅长设计游戏内场景的布置，比如各种物品的空间摆放。要求列出环境内存在哪些物品,物品摆放的坐标和物品,根据其使用环境的缩放系数,以及根据其形态和功能决定的物体朝向,返回数据的格式为一个表格。"},
    {"role": "user", "content": translation.text}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='| 物品 | 坐标 | 缩放系数 | 朝向 |\n|------|-------|-----------|------|\n| 圆桌 | (5, 5, 0) | 1 | 朝向北 |\n| 长沙发 | (10, 15, 0) | 1.2 | 朝向西 |\n| 咖啡机 | (12, 8, 0) | 0.8 | 朝向东 |\n| 吧台 | (18, 18, 0) | 1 | 朝向南 |\n| 植物 | (7, 3, 0) | 0.5 | 朝向西 |\n| 吊灯 | (5, 15, 3) | 1.5 | 朝向下 |\n根据场地尺寸20米长20米宽，设计了如上所示的咖啡馆环境。', role='assistant', function_call=None, tool_calls=None)
